In [1]:
import numpy as np

In [2]:
import pandas as pd
fields = ['WEIGHT', 'ABOID', 'AGEGRP', 'ATTSCH', 'BedRm', 'BFNMEMB', 'CFInc',
       'CFInc_AT', 'CfSize', 'CFSTAT', 'CIP2011', 'CIP2011_STEM_SUM',
       'Citizen', 'CitOth', 'CMA', 'CONDO', 'COW', 'DETH123', 'DPGRSUM',
       'DTYPE', 'EFDecile', 'EfDIMBM', 'EFInc', 'EFInc_AT', 'EfSize',
       'ETHDER', 'GENSTAT', 'HCORENEED_IND', 'HDGREE', 'HHInc',
       'HHInc_AT', 'HHMRKINC', 'HHSIZE', 'HHTYPE',
       'HLANO', 'IMMCAT5', 'IMMSTAT',
       'LFACT', 'LICO', 'LICO_AT', 'LOC_ST_RES', 'LoLIMA', 'LoLIMB',
       'LoMBM', 'LSTWRK', 'MarStH', 'MOB1', 'Mob5',
       'MTNNO', 'NOS', 'PKID0_1', 'PKID15_24', 'PKID2_5',
       'PKID25', 'PKID6_14', 'PKIDS', 'POB', 'POBF', 'POBM', 'PR', 'PR1',
       'PR5', 'PRIHM', 'REGIND', 'REPAIR', 'ROOMS', 'Sex', 'SHELCO',
       'SSGRAD', 'Tenur', 'VisMin', 'WRKACT']

dataset = pd.read_csv("pumf-2016-selected-features-No-MTEn-English.csv", usecols=fields)
dataset.head()
# removed market income (MrkInc, TotInc, TotInAT)

,WEIGHT,ABOID,AGEGRP,ATTSCH,BedRm,BFNMEMB,CFInc,CFInc_AT,CfSize,CFSTAT,...,PRIHM,REGIND,REPAIR,ROOMS,Sex,SHELCO,SSGRAD,Tenur,VisMin,WRKACT
0,37.120914,6,15,1,2,0,16,15,2,1,...,1,0,3,5,1,700,6,1,13,9
1,37.019784,6,18,1,3,0,13,12,1,6,...,1,0,1,5,1,900,6,1,13,1
2,37.041550,6,14,1,2,0,25,21,2,1,...,0,0,1,4,2,900,5,2,13,11
3,37.042280,6,11,1,3,0,28,25,4,2,...,1,0,1,8,2,2200,8,1,12,11
4,37.037277,6,12,1,4,0,29,25,4,2,...,0,0,1,10,1,2700,8,1,2,1


In [3]:
motherTongueDict = {}
motherTongueDict['1'] = 'No non-official languages'
motherTongueDict['2'] = 'Aboriginal languages'
motherTongueDict['3'] = 'Arabic'
motherTongueDict['4'] = 'Mandarin'
motherTongueDict['5'] = 'Cantonese'
motherTongueDict['6'] = 'Chinese languages'
motherTongueDict['7'] = 'German'
motherTongueDict['8'] = 'All other languages' #'Other Germanic languages'
motherTongueDict['9'] = 'All other languages' #'Greek'
motherTongueDict['10'] = 'Urdu'
motherTongueDict['11'] = 'Persian (Farsi)'
motherTongueDict['12'] = 'Other Indo-Iranian languages'
motherTongueDict['13'] = 'Italian'
motherTongueDict['14'] = 'Polish'
motherTongueDict['15'] = 'Portuguese'
motherTongueDict['16'] = 'Punjabi (Panjabi)'
motherTongueDict['17'] = 'Spanish'
motherTongueDict['18'] = 'All other languages' #'Ukrainian'
motherTongueDict['19'] = 'All other languages' #'Vietnamese'
motherTongueDict['20'] = 'Austro-Asiatic languages'
motherTongueDict['21'] = 'Other European languages'
motherTongueDict['22'] = 'Russian'
motherTongueDict['23'] = 'Slavic languages'
motherTongueDict['24'] = 'All other languages' #'Uralic languages'
motherTongueDict['25'] = 'Other Afro-Asiatic and African languages' #'Other Afro-Asiatic languages'
motherTongueDict['26'] = 'Tamil'
motherTongueDict['27'] = 'All other languages' #'Other Dravidian languages'
motherTongueDict['28'] = 'All other languages' #'Korean'
motherTongueDict['29'] = 'Other East and Southeast Asian languages'
motherTongueDict['30'] = 'Tagalog (Pilipino, Filipino)'
motherTongueDict['31'] = 'All other languages' #'Niger-Congo languages and other African languages'
motherTongueDict['32'] = 'All other languages' #'All other single languages'
motherTongueDict['88'] = 'Not available'

In [4]:
homeLanguageDict = {}
homeLanguageDict['1'] = 'No non-official languages'
homeLanguageDict['2'] = 'Aboriginal languages'
homeLanguageDict['3'] = 'Italian'
homeLanguageDict['4'] = 'Spanish'
homeLanguageDict['5'] = 'Portugese'
homeLanguageDict['6'] = 'German'
homeLanguageDict['7'] = 'Russian'
homeLanguageDict['8'] = 'Polish'
homeLanguageDict['9'] = 'Slavic languages'
homeLanguageDict['10'] = 'Other European languages'
homeLanguageDict['11'] = 'Arabic'
homeLanguageDict['12'] = 'Other Afro-Asiatic and African languages'
homeLanguageDict['13'] = 'Punjabi (Panjabi)'
homeLanguageDict['14'] = 'Urdu'
homeLanguageDict['15'] = 'Persian (Farsi)'
homeLanguageDict['16'] = 'Other Indo-Iranian languages'
homeLanguageDict['17'] = 'Cantonese'
homeLanguageDict['18'] = 'Mandarin'
homeLanguageDict['19'] = 'Chinese languages'
homeLanguageDict['20'] = 'Austro-Asiatic languages'
homeLanguageDict['21'] = 'Tagalog (Pilipino, Filipino)'
homeLanguageDict['22'] = 'Other East and Southeast Asian languages'
homeLanguageDict['23'] = 'Tamil'
homeLanguageDict['24'] = 'All other languages'
homeLanguageDict['88'] = 'Not available'

In [5]:
dataset = dataset[dataset.HLANO != 88]
dataset = dataset[dataset.MTNNO != 88]
# class label: home language part A - first language write in component
homeLang = dataset['HLANO']
# class label: mother tongue part A - first language write in component
motherTongue = dataset['MTNNO']
fields.remove('HLANO')
fields.remove('MTNNO')
homeLanguageMapped = homeLang.apply(lambda x: homeLanguageDict[str(x)])
motherTongueMapped = motherTongue.apply(lambda x: motherTongueDict[str(x)])
languageShift = homeLanguageMapped.ne(motherTongueMapped)

fields.remove('WEIGHT')
x = dataset[fields]
weights = dataset["WEIGHT"]

In [6]:
homeLanguageMapped

0                        No non-official languages
1                        No non-official languages
2                        No non-official languages
3                        No non-official languages
4                        No non-official languages
5                        No non-official languages
6                        No non-official languages
9                        No non-official languages
11        Other Afro-Asiatic and African languages
12                       No non-official languages
13                                       Portugese
14                                       Portugese
15                       No non-official languages
16                       No non-official languages
17                       No non-official languages
18                    Tagalog (Pilipino, Filipino)
19                                Slavic languages
20                                Slavic languages
21                       No non-official languages
22                             

In [7]:
motherTongueMapped

0            No non-official languages
1            No non-official languages
2            No non-official languages
3                            Cantonese
4                             Mandarin
5            No non-official languages
6            No non-official languages
9            No non-official languages
11                 All other languages
12           No non-official languages
13                          Portuguese
14                          Portuguese
15           No non-official languages
16           No non-official languages
17           No non-official languages
18        Tagalog (Pilipino, Filipino)
19                    Slavic languages
20                    Slavic languages
21           No non-official languages
22                           Cantonese
23                           Cantonese
24                                Urdu
25           No non-official languages
26           No non-official languages
27                              German
28                       

In [8]:
languageShift

0         False
1         False
2         False
3          True
4          True
5         False
6         False
9         False
11         True
12        False
13         True
14         True
15        False
16        False
17        False
18        False
19        False
20        False
21        False
22        False
23        False
24        False
25        False
26        False
27        False
28        False
29        False
30        False
31        False
32        False
          ...  
387787    False
387788    False
387789    False
387790    False
387791     True
387792    False
387793    False
387794     True
387795     True
387796    False
387797    False
387798     True
387799    False
387800    False
387801    False
387802     True
387803    False
387804    False
387805    False
387806    False
387807    False
387808    False
387809    False
387810    False
387811     True
387812     True
387813    False
387814    False
387815    False
387816    False
Length: 379075, dtype: b

In [9]:
x

,ABOID,AGEGRP,ATTSCH,BedRm,BFNMEMB,CFInc,CFInc_AT,CfSize,CFSTAT,CIP2011,...,PRIHM,REGIND,REPAIR,ROOMS,Sex,SHELCO,SSGRAD,Tenur,VisMin,WRKACT
0,6,15,1,2,0,16,15,2,1,5,...,1,0,3,5,1,700,6,1,13,9
1,6,18,1,3,0,13,12,1,6,5,...,1,0,1,5,1,900,6,1,13,1
2,6,14,1,2,0,25,21,2,1,11,...,0,0,1,4,2,900,5,2,13,11
3,6,11,1,3,0,28,25,4,2,7,...,1,0,1,8,2,2200,8,1,12,11
4,6,12,1,4,0,29,25,4,2,6,...,0,0,1,10,1,2700,8,1,2,1
5,6,14,1,4,0,15,13,1,6,13,...,1,0,1,6,1,1300,4,1,13,9
6,6,1,9,4,0,29,26,4,4,99,...,0,0,1,11,1,2658,99,1,12,99
9,6,9,4,2,0,11,11,2,1,3,...,1,0,2,4,2,900,6,2,13,9
11,6,9,4,1,0,13,13,2,1,10,...,0,0,1,3,1,900,11,2,3,6
12,6,9,1,1,0,9,9,2,1,4,...,0,0,1,4,1,800,8,2,13,6


In [10]:
# discretization

def discretify(dataset, quantiles=10, target_fields=('SHELCO', 'MrkInc', 'TotInc', 'TotInc_AT', 'VALUE'),
               invalid_quantile = 88):
    """Convert target fields in dataset to quantile categories."""
    
    invalid_values = {
        'SHELCO': (),
        'MrkInc': (88888888, 99999999),
        'TotInc': (88888888, 99999999),
        'TotInc_AT': (88888888, 99999999),
        'VALUE': (88888888, 99999999)
    }
    
    dataset_copy = dataset.copy()

    for continuous in target_fields:
        if continuous in dataset_copy:
            print(continuous)
            values = []
            quantile_values = pd.qcut(dataset_copy[continuous].rank(method='first'), quantiles, False)

            for value, quantile in zip(dataset_copy[continuous], quantile_values):
                if value in invalid_values[continuous]:
                    values.append(invalid_quantile)
                else:
                    values.append(quantile)

            dataset_copy[continuous] = values
    
    return dataset_copy


x = discretify(x)

x

SHELCO


,ABOID,AGEGRP,ATTSCH,BedRm,BFNMEMB,CFInc,CFInc_AT,CfSize,CFSTAT,CIP2011,...,PRIHM,REGIND,REPAIR,ROOMS,Sex,SHELCO,SSGRAD,Tenur,VisMin,WRKACT
0,6,15,1,2,0,16,15,2,1,5,...,1,0,3,5,1,1,6,1,13,9
1,6,18,1,3,0,13,12,1,6,5,...,1,0,1,5,1,3,6,1,13,1
2,6,14,1,2,0,25,21,2,1,11,...,0,0,1,4,2,3,5,2,13,11
3,6,11,1,3,0,28,25,4,2,7,...,1,0,1,8,2,8,8,1,12,11
4,6,12,1,4,0,29,25,4,2,6,...,0,0,1,10,1,9,8,1,2,1
5,6,14,1,4,0,15,13,1,6,13,...,1,0,1,6,1,5,4,1,13,9
6,6,1,9,4,0,29,26,4,4,99,...,0,0,1,11,1,9,99,1,12,99
9,6,9,4,2,0,11,11,2,1,3,...,1,0,2,4,2,3,6,2,13,9
11,6,9,4,1,0,13,13,2,1,10,...,0,0,1,3,1,3,11,2,3,6
12,6,9,1,1,0,9,9,2,1,4,...,0,0,1,4,1,2,8,2,13,6


In [11]:
weights

0         37.120914
1         37.019784
2         37.041550
3         37.042280
4         37.037277
5         37.041550
6         37.041550
9         37.041550
11        37.012589
12        37.041550
13        37.041550
14        37.041550
15        37.041550
16        37.041550
17        37.041550
18        37.019784
19        37.037277
20        37.037277
21        37.041550
22        37.037277
23        37.037277
24        37.042280
25        37.041550
26        37.041550
27        37.037277
28        37.037277
29        37.041550
30        37.037277
31        37.041550
32        37.041550
            ...    
387787    37.041550
387788    37.041550
387789    37.041550
387790    37.037277
387791    37.037277
387792    37.037277
387793    37.037277
387794    37.037277
387795    37.037277
387796    37.041550
387797    37.037277
387798    37.037277
387799    37.037277
387800    37.041550
387801    37.037277
387802    37.042280
387803    37.041550
387804    37.041550
387805    37.041550


In [12]:
!pip3 install scikit-learn==0.22

  Using cached https://files.pythonhosted.org/packages/2e/d0/860c4f6a7027e00acff373d9f5327f4ae3ed5872234b3cbdd7bcb52e5eff/scikit_learn-0.22-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/d2/ab/43e678759326f728de861edbef34b8e2ad1b1490505f20e0d1f0716c3bf4/numpy-1.17.4-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/28/5c/cf6a2b65a321c4a209efcdf64c2689efae2cb62661f8f6f4bb28547cf1bf/joblib-0.14.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/f0/7a/579f4659bb488f19f6b458a003b36b86bacaedcb28530f8d25cb153fc5cd/scipy-1.4.0-cp36-cp36m-manylinux1_x86_64.whl


In [13]:
import sys 
sys.version

'3.6.9 (default, Nov  7 2019, 10:44:02) \n[GCC 8.3.0]'

In [ ]:
from sklearn import datasets

In [ ]:
from sklearn.naive_bayes import CategoricalNB
classifier = CategoricalNB()

In [ ]:
from sklearn.model_selection import train_test_split
weights_train, weights_test, x_train, x_test, languageShift_train, languageShift_test = train_test_split(weights, x, languageShift)

In [ ]:
x_train[(x_train < 0).any(1)]

In [ ]:
classifier.fit(x_train, languageShift_train, sample_weight=weights_train.tolist())

In [ ]:
languageShift_pred = classifier.predict(x_test)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

The Confusion Matrix indicates, given the predicted language (row names), for how many records did the algorith match the actual language. The classification report and accuracy score are other ways to evaluate how well the classification algorithm performed.

In [ ]:
confusionMatrixLanguageShift = pd.DataFrame(
    confusion_matrix(languageShift_test,languageShift_pred), 
    index=['False', 'True'], 
    columns=['False', 'True'], 
)

In [ ]:
print(confusionMatrixLanguageShift)
print(classification_report(languageShift_test,languageShift_pred,target_names=['False', 'True']))
print(accuracy_score(languageShift_test, languageShift_pred))

In [ ]:
!pip install tabulate

In [1]:
classifier.classes_

In [ ]:
from tabulate import tabulate
importances = classifier.feature_log_prob_
print(importances)
#headers = ["name", "prob of feature given class: False", "prob of feature given class: True"]
#values = sorted(zip(x_train.columns, np.exp(importances[0]), np.exp(importances[1])), key=lambda x: (x[2] * -1, x[1] * -1))
#print(tabulate(values, headers, tablefmt="plain"))

In [ ]:
!pip install bokeh

In [ ]:
from bokeh.io import push_notebook, show, output_notebook
from bokeh.layouts import row
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, HoverTool

output_notebook()
import numpy as np

In [ ]:
#from sklearn.tree import export_graphviz
#estimator = classifier.estimators_[5]
# Creates dot file named randomForestTreeSample.dot
#export_graphviz(estimator, 
#                out_file='randomForestTreeSample.dot', 
#                feature_names = list(x.columns),
#                class_names = ['False', 'True'],
#                rounded = True, proportion = False, 
#                precision = 2, filled = True,
#                max_depth=5)